In [1]:
import sys
sys.path.append("/Colorization/src")  # Append path to src
import time
import numpy as np

In [2]:
from train import get_parser, train
run_cmd = './src/train.py eccv16'  # Training
run_cmd = './src/train.py eccv16 --batch-per-epoch 10 --val-batch 10'  # Testing
run_cmd = './src/train.py eccv16_pretrained --batch-per-epoch 10 --val-batch 1'  # Testing
run_cmd = './src/train.py eccv16_half --batch-size 256 --val-batch-size 256 --batch-per-epoch 30 --val-batch 10'  # Testing
run_cmd = './src/train.py eccv16_half --batch-size 256 --val-batch-size 256 --val-batch 8'  # Training
run_cmd = './src/train.py eccv16_half --loss-func MSELoss --batch-size 256 --val-batch-size 256 --val-batch 8'  # Training
run_cmd = './src/train.py eccv16_half --ckpt-filepath /Colorization/checkpoints/eccv16_half-20210313_235419/cp-020-0.3544.ckpt.pth --num-epochs 30 --batch-size 256 --val-batch-size 256 --val-batch 8'  # Resume Training
run_cmd = './src/train.py eccv16_half --ckpt-filepath /Colorization/checkpoints/eccv16_half-20210313_235419/cp-020-0.3544.ckpt.pth --num-epochs 30 --color-vivid-gamma 3.0 --batch-size 256 --val-batch-size 256 --val-batch 8'  # Resume Training, gamma=3.0
run_cmd = './src/train.py eccv16_half --color-vivid-gamma 1.0 --batch-size 256 --val-batch-size 256 --val-batch 8'  # Training, gamma=1.0

Matplotlib created a temporary config/cache directory at /tmp/matplotlib-1dg9z4nf because the default path (/.config/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


In [3]:
parser = get_parser()
train_args = parser.parse_args(run_cmd.split()[1:])
train(train_args)


Num GPUs Available: 1

Found existing prior probs, won't regenerate it
Train/Val Dataset "train2017" loaded: 118060 samples, 462 batches
Train/Val Dataset "val2017" loaded: 5000 samples, 20 batches
Predict Dataset "val2017" loaded: 6 samples, 6 batches


  0%|          | 0/462 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:3455: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode)
  0%|          | 0/6 [00:00<?, ?it/s]

Epoch 1/20: train_loss=14317751.0	val_loss=13162118.0	val_auc=0.5569


  0%|          | 0/462 [00:00<?, ?it/s]

Checkpoint saved to /Colorization/checkpoints/eccv16_half-20210317_111335/cp-001-0.5569.ckpt.pth


...

Checkpoint saved to /Colorization/checkpoints/eccv16_half-20210317_111335/cp-019-0.3963.ckpt.pth


  0%|          | 0/6 [00:00<?, ?it/s]

Epoch 20/20: train_loss=6334150.0	val_loss=10286614.0	val_auc=0.3967


100%|██████████| 6/6 [00:01<00:00,  3.82it/s]


Checkpoint saved to /Colorization/checkpoints/eccv16_half-20210317_111335/cp-020-0.3967.ckpt.pth
Training finished!


## Visualize val images

In [2]:
import argparse

In [3]:
run_cmd = './src/train.py -vi 10 -vi 23 --val-img-idx 2'  # Training

In [4]:
parser = argparse.ArgumentParser()
parser.add_argument('-vi', '--val-img-idx', type=int, action='append', default=[1, 5, 9])
args = parser.parse_args(run_cmd.split()[1:])

In [37]:
sorted(args.val_img_idx)

[1, 2, 5, 9, 10, 23]

In [21]:
import os, glob
img_paths = sorted(glob.glob(os.path.join('/Colorization/data/val2017', '*.jpg')))

In [30]:
img_paths[30]

'/Colorization/data/val2017/000000002532.jpg'

In [33]:
len(args.val_img_idx)

6

In [6]:
args.hw_resize = 256

In [7]:
args

Namespace(hw_resize=256, val_img_idx=[1, 5, 9, 10, 23, 2])